# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    #ACHAR UM JEITO DE RETIRAR OS @S E OS NOMES QUE VEM SEGUIDOS DELES
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/mac/Desktop/ciencia-dos-dados-p1-main 2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'iFood.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,me dá cupom de desconto @ifood,1
1,é hoje!\na partir das 18:30 estaremos 100% onl...,0
2,todo dia eu entro no ifood pra ver se ele tá m...,0
3,eu n to acreditando que o entregador do ifood ...,1
4,preciso apresentar essa música p uns parceiro ...,0


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@matheusdevdd @rclorandi @rolealeatorio @ifood...
1,@taldoalvesss eu fico parecendo um entregador ...
2,@gdallaru @ifood acordou engraçadinho e com a ...
3,"pedido mais rápido q já fiz no ifood, 20 min f..."
4,"ele não pega ninguém, só leva p dar rolê, ifoo..."


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

O produto escolhido foi o serviço do iFood. Para a classificação dos tweets, consideramos relevantes aqueles que forneciam opiniões de clientes, com claras críticas ou elogios.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
Limpo = []
for i in range (len(train.Treinamento)):
    Limpo.append(cleanup(train.Treinamento[i].lower()))
train["Limpo"] = pd.Series(Limpo)

tudo = ""
for tweet in train.Limpo:
    tudo = tudo + tweet
tudo_separado = tudo.split()

In [8]:
relevante = train.loc[(train.Classificação == 1), "Limpo"]
irrelevante = train.loc[(train.Classificação == 0), "Limpo"]

relevante_texto = ""

for tweet in relevante:
    relevante_texto = relevante_texto + tweet

irrelevante_texto = ""
for tweet in irrelevante:
    irrelevante_texto = irrelevante_texto + tweet

relevante_separado = relevante_texto.split()
irrelevante_separado = irrelevante_texto.split()

prob_r = len(relevante_separado)/len(tudo_separado)
prob_i = len(irrelevante_separado)/len(tudo_separado)
print(prob_r)
print(prob_i)



0.3318951211417192
0.6682708264188516


In [17]:
serie_relevante = pd.Series(relevante_separado)
tabela_relevante = serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)

print(serie_relevante)
print(len(tabela_relevante))
for i in range(len(tabela_relevante) - 1):
    if tabela_relevante[i] == tabela_relevante[i+1]:
        tabela_relevante[i] = 1

serie_irrelevante = pd.Series(irrelevante_separado)
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)


0             me
1             dá
2          cupom
3             de
4       desconto
          ...   
1995       menos
1996          de
1997         uma
1998        hora
1999          🗣️
Length: 2000, dtype: object
811


In [21]:
def classifica_tweets(tweet):
    ProbR = 0.3318951211417192
    ProbI = 0.6682708264188516
    probTweetDadoR = 1
    probTweetDadoI = 1
    tweet_separado = tweet.split()
    for i in range(len(tweet_separado)):
        if tweet_separado[i] in tabela_relevante_relativa:
            probTweetDadoR = probTweetDadoR*(tabela_relevante[tweet_separado[i]]+1)/(len(serie_relevante) + len(tabela_relevante))
        else:
            probTweetDadoR = probTweetDadoR*(0+1)/(len(serie_relevante) + len(tabela_relevante))
        if tweet_separado[i] in tabela_irrelevante_relativa:
            probTweetDadoI = probTweetDadoI*(tabela_irrelevante[tweet_separado[i]]+1)/(len(serie_irrelevante) + len(tabela_irrelevante))
        else:
            probTweetDadoI = probTweetDadoI*(0+1)/(len(serie_irrelevante) + len(tabela_irrelevante))
    ProbRDadoTweet = probTweetDadoR*ProbR
    ProbIDadoTweet = probTweetDadoI*ProbI
    if ProbRDadoTweet > ProbIDadoTweet:
        return("O tweet '{0}' é relevante".format(tweet))
    else:
        return("O tweet '{0}' é irrelevante".format(tweet))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [22]:
teste_limpo = []
probTweetDadoR = 1
probTweetDadoI = 1


for i in range (len(test.Teste)):
    teste_limpo.append(cleanup(test.Teste[i].lower()))
    
for tweet in teste_limpo:
    print(classifica_tweets(tweet))

O tweet '@matheusdevdd @rclorandi @rolealeatorio @ifood tirei do seu cuzinho inclusive arrombado' é relevante
O tweet '@taldoalvesss eu fico parecendo um entregador do ifood' é irrelevante
O tweet '@gdallaru @ifood acordou engraçadinho e com a foto desse ombro com marrinha de porra nenhuma né @victorratz' é irrelevante
O tweet 'pedido mais rápido q já fiz no ifood 20 min foi o tempo entre eu fazer o pedido e abrir a marmita exatos 20 min' é irrelevante
O tweet 'ele não pega ninguém só leva p dar rolê ifood ifood de mulher kkkkkkkkkk 🤌🏾' é irrelevante
O tweet 'o menino pedindo pra eu mandar um ifood pra ele a minha cara https//tco/gunzrcnebg' é irrelevante
O tweet '@rclorandi @tricktrickb @rolealeatorio @ifood se ele já até ligou na operadora acha que não tem bo tirou a informação do cu né' é relevante
O tweet '@matheusdevdd @rclorandi @rolealeatorio @ifood eu não ganho dinheiro com rede social logo não tenho meta de seguidor logo se pra você ter seguidor é importante  até nisso você fr

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**